In [1]:
import os
import math
import json
from tqdm.auto import tqdm
from collections import Counter

import torch
import torch.nn as nn
import torch.utils.data
from torch.utils.data import Dataset
import torch.nn.functional as F

from src.config import Config

In [2]:
def remove_punc(string):
    punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
    no_punct = ""
    for char in string:
        if char not in punctuations:
            no_punct = no_punct + char  # space is also a character
    return no_punct.lower()

In [3]:
corpus_movie_conv = os.path.join(Config.FILES["CORNELL_MOVIE_DIR"], "movie_conversations.txt")
corpus_movie_lines = os.path.join(Config.FILES["CORNELL_MOVIE_DIR"], "movie_lines.txt")
max_len = 25

with open(corpus_movie_conv, "r") as c:
    conv = c.readlines()
    
with open(corpus_movie_lines, "r") as l:
    lines = l.readlines()
    
lines_dic = {}
for line in lines:
    objects = line.split(" +++$+++ ")
    lines_dic[objects[0]] = objects[-1]

In [4]:
pairs = []
for con in tqdm(conv):
    ids = eval(con.split(" +++$+++ ")[-1])
    for i in range(len(ids)):
        qa_pairs = []
        
        if i==len(ids)-1:
            break
        
        first = remove_punc(lines_dic[ids[i]].strip())
        second = remove_punc(lines_dic[ids[i+1]].strip())
        qa_pairs.append(first.split()[:max_len])
        qa_pairs.append(second.split()[:max_len])
        pairs.append(qa_pairs)
        
word_freq = Counter()
for pair in tqdm(pairs):
    word_freq.update(pair[0])
    word_freq.update(pair[1])
    
min_word_freq = 5
words = [w for w in word_freq.keys() if word_freq[w] > min_word_freq]
word_map = {k: v + 1 for v, k in enumerate(words)}
word_map["<unk>"] = len(word_map) + 1
word_map["<start>"] = len(word_map) + 1
word_map["<end>"] = len(word_map) + 1
word_map["<pad>"] = 0

print("Total words are: {}".format(len(word_map)))

  0%|          | 0/83097 [00:00<?, ?it/s]

  0%|          | 0/221616 [00:00<?, ?it/s]

Total words are: 18243


In [5]:
fname = os.path.join(Config.FILES["PREPROCESS_DIR"], "WORDMAP_corpus.json")
with open(fname, "w") as j:
    json.dump(word_map, j)

In [6]:
def encode_question(words, word_map):
    enc_c = [word_map.get(word, word_map["<unk>"]) for word in words] + [word_map["<pad>"]] * (max_len-len(words))
    
    return enc_c


def encode_reply(words, word_map):
    enc_c = [word_map["<start>"]] + [word_map.get(word, word_map["<unk>"]) for word in words] + \
        [word_map["<end>"]] + [word_map["<pad>"]] * (max_len-len(words))
        
    return enc_c

In [8]:
pairs_encoded = []
for pair in pairs:
    qus = encode_question(pair[0], word_map)
    ans = encode_reply(pair[1], word_map)
    pairs_encoded.append([qus, ans])

In [10]:
fname = os.path.join(Config.FILES["PREPROCESS_DIR"], "pairs_encoded.json")
with open(fname, "w") as p:
    json.dump(pairs_encoded, p)

## Transform into Dataset Iterators

In [11]:
class Dataset(Dataset):
    def __init__(self):
        self.pairs = json.load(open(os.path.join(Config.FILES["PREPROCESS_DIR"], "pairs_encoded.json")))
        self.dataset_size = len(self.pairs)
        
    
    def __getitem__(self, i):
        question = torch.LongTensor(self.pairs[i][0])
        reply = torch.LongTensor(self.pairs[i][1])
        
        return question, reply
    
    
    def __len__(self):
        return self.dataset_size

In [12]:
train_loader = torch.utils.data.DataLoader(Dataset(),
                                           batch_size=100,
                                           shuffle=True,
                                           pin_memory=True)

In [40]:
def create_masks(question, reply_input, reply_target):
    def subsequent_mask(size):
        mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype=torch.uint8)
        return mask.unsqueeze(0)
    
    question_mask = question!=0
    question_mask = question_mask.to(device)
    question_mask = question_mask.unsqueeze(1).unsqueeze(1)
    
    reply_input_mask = reply_input!=0
    reply_input_mask = reply_input_mask.unsqueeze(1)
    reply_input_mask = reply_input_mask & subsequent_mask(reply_input.size(-1)).type_as(reply_input_mask.data)
    reply_input_mask = reply_input_mask.unsqueeze(1)
    reply_target_mask = reply_target!=0
    
    return question_mask, reply_input_mask, reply_target_mask

## Create Transformers Model

In [41]:
class Embeddings(nn.Module):
    def __init__(self, vocab_size, d_model, max_len=50):
        super(Embeddings, self).__init__()
        self.d_model = d_model
        self.dropout = nn.Dropout(0.1)
        self.embed = nn.Embedding(vocab_size, d_model)
        self.pe = self.create_positional_encoding(max_len, self.d_model)
        self.dropout = nn.Dropout(0.1)
        
        
    def create_positional_encoding(self, max_len, d_model):
        pe = torch.zeros(max_len, d_model).to(device)
        for pos in range(max_len):
            for i in range(0, d_model, 2):
                pe[pos, i] = math.sin(pos / (10000 ** ((2*i)/d_model)))
                pe[pos, i+1] = math.cos(pos / (10000 ** ((2*(i+1))/d_model)))
        pe = pe.unsqueeze(0)
        
        return pe
    
    
    def forward(self, encoded_words):
        embedding = self.embed(encoded_words) * math.sqrt(self.d_model)
        embedding += self.pe[:, :embedding.size(1)]
        embedding = self.dropout(embedding)
        
        return embedding

In [53]:
class MultiHeadAttention(nn.Module):
    def __init__(self, heads, d_model):
        super(MultiHeadAttention, self).__init__()
        assert d_model % heads == 0
        self.d_k = d_model // heads
        self.heads = heads
        self.dropout = nn.Dropout(0.1)
        self.query = nn.Linear(d_model, d_model)
        self.key = nn.Linear(d_model, d_model)
        self.value = nn.Linear(d_model, d_model)
        self.concat = nn.Linear(d_model, d_model)
        
        
    def forward(self, query, key, value, mask):
        query = self.query(query)
        key = self.key(key)
        value = self.value(value)
        
        query = query.view(query.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)
        key = key.view(key.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)
        value = value.view(value.shape[0], -1, self.heads, self.d_k).permute(0, 2, 1, 3)
        
        scores = torch.matmul(query, key.permute(0,1,3,2)) / math.sqrt(query.size(-1))
        scores = scores.masked_fill(mask==0, -1e9)
        weights = F.softmax(scores, dim=-1)
        weights = self.dropout(weights)
        
        context = torch.matmul(weights, value)
        context = context.permute(0,2,1,3).contiguous().view(context.shape[0], -1, self.heads * self.d_k)
        
        interacted = self.concat(context)
        
        return interacted

In [54]:
class FeedForward(nn.Module):
    def __init__(self, d_model, middle_dim=2048):
        super(FeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, middle_dim)
        self.fc2 = nn.Linear(middle_dim, d_model)
        self.dropout = nn.Dropout(0.1)
        
        
    def forward(self, x):
        out = F.relu(self.fc1(x))
        out = self.fc2(self.dropout(out))
        return out

In [55]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, heads):
        super(EncoderLayer, self).__init__()
        self.layernorm = nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout = nn.Dropout(0.1)
        
        
    def forward(self, embeddings, mask):
        interacted = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, mask))
        interacted = self.layernorm(interacted + embeddings)
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        encoded = self.layernorm(feed_forward_out + interacted)
        
        return encoded

In [64]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, heads):
        super(DecoderLayer, self).__init__()
        self.layernorm = nn.LayerNorm(d_model)
        self.self_multihead = MultiHeadAttention(heads, d_model)
        self.src_multihead = MultiHeadAttention(heads, d_model)
        self.feed_forward = FeedForward(d_model)
        self.dropout = nn.Dropout(0.1)
        
        
    def forward(self, embeddings, encoded, src_mask, target_mask):
        query = self.dropout(self.self_multihead(embeddings, embeddings, embeddings, target_mask))
        query = self.layernorm(query + embeddings)
        interacted = self.dropout(self.src_multihead(query, encoded, encoded, src_mask))
        interacted = self.layernorm(interacted + query)
        feed_forward_out = self.dropout(self.feed_forward(interacted))
        decoded = self.layernorm(feed_forward_out + interacted)
        
        return decoded

In [65]:
class Transformer(nn.Module):
    def __init__(self, d_model, heads, num_layers, word_map):
        super(Transformer, self).__init__()
        
        self.d_model = d_model
        self.vocab_size = len(word_map)
        self.embed = Embeddings(self.vocab_size, d_model)
        self.encoder = nn.ModuleList([EncoderLayer(d_model, heads) for _ in range(num_layers)])
        self.decoder = nn.ModuleList([DecoderLayer(d_model, heads) for _ in range(num_layers)])
        self.logit = nn.Linear(d_model, self.vocab_size)
        
        
    def encode(self, src_words, src_mask):
        src_embeddings = self.embed(src_words)
        for layer in self.encoder:
            src_embeddings = layer(src_embeddings, src_mask)
        return src_embeddings
    
    
    def decode(self, target_words, target_mask, src_embeddings, src_mask):
        tgt_embeddings = self.embed(target_words)
        for layer in self.decoder:
            tgt_embeddings = layer(tgt_embeddings, src_embeddings, src_mask, target_mask)
        return tgt_embeddings
        
        
    def forward(self, src_words, src_mask, target_words, target_mask):
        encoded = self.encode(src_words, src_mask)
        decoded = self.decode(target_words, target_mask, encoded, src_mask)
        out = F.log_softmax(self.logit(decoded), dim = 2)
        return out

In [66]:
class AdamWarmup:
    def __init__(self, model_size, warmup_steps, optimizer):
        
        self.model_size = model_size
        self.warmup_steps = warmup_steps
        self.optimizer = optimizer
        self.current_step = 0
        self.lr = 0
        
        
    def get_lr(self):
        return self.model_size ** (-0.5) * min(self.current_step ** (-0.5), self.current_step * self.warmup_steps ** (-1.5))
        
        
    def step(self):
        # Increment the number of steps each time we call the step function
        self.current_step += 1
        lr = self.get_lr()
        for param_group in self.optimizer.param_groups:
            param_group['lr'] = lr
        # update the learning rate
        self.lr = lr
        self.optimizer.step()

In [67]:
class LossWithLS(nn.Module):
    def __init__(self, size, smooth):
        super(LossWithLS, self).__init__()
        self.criterion = nn.KLDivLoss(size_average=False, reduce=False)
        self.confidence = 1.0 - smooth
        self.smooth = smooth
        self.size = size
        
        
    def forward(self, prediction, target, mask):
        """
        prediction of shape: (batch_size, max_words, vocab_size)
        target and mask of shape: (batch_size, max_words)
        """
        prediction = prediction.view(-1, prediction.size(-1))   # (batch_size * max_words, vocab_size)
        target = target.contiguous().view(-1)   # (batch_size * max_words)
        mask = mask.float()
        mask = mask.view(-1)       # (batch_size * max_words)
        labels = prediction.data.clone()
        labels.fill_(self.smooth / (self.size - 1))
        labels.scatter_(1, target.data.unsqueeze(1), self.confidence)
        loss = self.criterion(prediction, labels)    # (batch_size * max_words, vocab_size)
        loss = (loss.sum(1) * mask).sum() / mask.sum()
        return loss

In [68]:
d_model = 512
heads = 8
num_layers = 3
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
epochs = 10

fname = os.path.join(Config.FILES["PREPROCESS_DIR"], "WORDMAP_corpus.json")
with open(fname, "r") as j:
    word_map = json.load(j)
    
transformer = Transformer(d_model=d_model, heads=heads, num_layers=num_layers, word_map=word_map)
transformer = transformer.to(device)
adam_optimizer = torch.optim.Adam(transformer.parameters(), lr=0, betas=(0.9, 0.98), eps=1e-9)
transformer_optimizer = AdamWarmup(model_size = d_model, warmup_steps = 4000, optimizer = adam_optimizer)
criterion = LossWithLS(len(word_map), 0.1)

## Model Training

In [69]:
def train(train_loader, transformer, criterion, epoch):
    transformer.train()
    sum_loss = 0
    count = 0
    for i, (question, reply) in enumerate(train_loader):
        
        samples = question.shape[0]

        # Move to device
        question = question.to(device)
        reply = reply.to(device)

        # Prepare Target Data
        reply_input = reply[:, :-1]
        reply_target = reply[:, 1:]

        # Create mask and add dimensions
        question_mask, reply_input_mask, reply_target_mask = create_masks(question, reply_input, reply_target)

        # Get the transformer outputs
        out = transformer(question, question_mask, reply_input, reply_input_mask)

        # Compute the loss
        loss = criterion(out, reply_target, reply_target_mask)
        
        # Backprop
        transformer_optimizer.optimizer.zero_grad()
        loss.backward()
        transformer_optimizer.step()
        
        sum_loss += loss.item() * samples
        count += samples
        
        if i % 100 == 0:
            print("Epoch [{}][{}/{}]\tLoss: {:.3f}".format(epoch, i, len(train_loader), sum_loss/count))

In [70]:
def evaluate(transformer, question, question_mask, max_len, word_map):
    """
    Performs Greedy Decoding with a batch size of 1
    """
    rev_word_map = {v: k for k, v in word_map.items()}
    transformer.eval()
    start_token = word_map['<start>']
    encoded = transformer.encode(question, question_mask)
    words = torch.LongTensor([[start_token]]).to(device)
    
    for step in range(max_len - 1):
        size = words.shape[1]
        target_mask = torch.triu(torch.ones(size, size)).transpose(0, 1).type(dtype=torch.uint8)
        target_mask = target_mask.to(device).unsqueeze(0).unsqueeze(0)
        decoded = transformer.decode(words, target_mask, encoded, question_mask)
        predictions = transformer.logit(decoded[:, -1])
        _, next_word = torch.max(predictions, dim = 1)
        next_word = next_word.item()
        if next_word == word_map['<end>']:
            break
        words = torch.cat([words, torch.LongTensor([[next_word]]).to(device)], dim = 1)   # (1,step+2)
        
    # Construct Sentence
    if words.dim() == 2:
        words = words.squeeze(0)
        words = words.tolist()
        
    sen_idx = [w for w in words if w not in {word_map['<start>']}]
    sentence = ' '.join([rev_word_map[sen_idx[k]] for k in range(len(sen_idx))])
    
    return sentence

In [72]:
for epoch in range(epochs):
    train(train_loader, transformer, criterion, epoch)
    
    state = {"epoch": epoch, "transformer": transformer, "transformer_optimizer": transformer_optimizer}
    model_file = os.path.join(Config.FILES["MODEL_DATA_DIR"], "transformers-custom", ("checkpoint_" + str(epoch) + ".pth.tar"))
    torch.save(state, model_file)

Epoch [0][0/2217]	Loss: 4.260
Epoch [0][100/2217]	Loss: 4.419
Epoch [0][200/2217]	Loss: 4.414
Epoch [0][300/2217]	Loss: 4.414
Epoch [0][400/2217]	Loss: 4.412
Epoch [0][500/2217]	Loss: 4.409
Epoch [0][600/2217]	Loss: 4.411
Epoch [0][700/2217]	Loss: 4.411
Epoch [0][800/2217]	Loss: 4.409
Epoch [0][900/2217]	Loss: 4.409
Epoch [0][1000/2217]	Loss: 4.409
Epoch [0][1100/2217]	Loss: 4.408
Epoch [0][1200/2217]	Loss: 4.408
Epoch [0][1300/2217]	Loss: 4.408
Epoch [0][1400/2217]	Loss: 4.407
Epoch [0][1500/2217]	Loss: 4.407
Epoch [0][1600/2217]	Loss: 4.407
Epoch [0][1700/2217]	Loss: 4.407
Epoch [0][1800/2217]	Loss: 4.407
Epoch [0][1900/2217]	Loss: 4.407
Epoch [0][2000/2217]	Loss: 4.406
Epoch [0][2100/2217]	Loss: 4.407
Epoch [0][2200/2217]	Loss: 4.406
Epoch [1][0/2217]	Loss: 4.459
Epoch [1][100/2217]	Loss: 4.315
Epoch [1][200/2217]	Loss: 4.317
Epoch [1][300/2217]	Loss: 4.311
Epoch [1][400/2217]	Loss: 4.308
Epoch [1][500/2217]	Loss: 4.309
Epoch [1][600/2217]	Loss: 4.310
Epoch [1][700/2217]	Loss: 4.310

## Testing

In [73]:
fname = os.path.join(Config.FILES["MODEL_DATA_DIR"], "transformers-custom", "checkpoint_9.pth.tar")
checkpoint = torch.load(fname)
transformer = checkpoint["transformer"]

In [77]:
while(1):
    question = input("Question: ") 
    if question == 'quit':
        break
    max_len = 100
    enc_qus = [word_map.get(word, word_map["<unk>"]) for word in question.split()]
    question = torch.LongTensor(enc_qus).to(device).unsqueeze(0)
    question_mask = (question!=0).to(device).unsqueeze(1).unsqueeze(1)  
    sentence = evaluate(transformer, question, question_mask, int(max_len), word_map)
    print(sentence)

i dont know
i dont know
i dont know
i dont know
i dont know
i dont know
i dont know
i dont know
i dont know
i dont know
i dont know
i dont know
i dont know
i dont know
i dont know
i dont know
